In [7]:
#external libraries
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point,Polygon
import matplotlib.pyplot as plt
import folium
from folium.plugins import TimestampedGeoJson
from datetime import datetime

import matplotlib as mpl
import matplotlib.colors as mcolors

%matplotlib inline
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
#local imports
from src.DataLoader import gdfLoader,ReceiverLoader,RSSIDataLoader
from src.visualization import MapCreation
from func.utils.getMarkerDictList import getMarkerDictList
from func.FoliumFeatureCreation.RawDataFeatures import RawDataFeatures


In [9]:
MapDataPath="data/cartePalaiseau.csv"
ModuleMapPath="data/carteModulePalaiseau.csv"
Data_Path="data/manip1.1.csv"

saving_output=False

sampling_time=2000#ms #si supérieur à 2000ms il faut implémenter la moyenne dans la liste des rssi dans marker dict list

### Data loading


In [10]:
#Map Data
map_gdf=gdfLoader(MapDataPath)
#marker data
marker_gdf=ReceiverLoader(ModuleMapPath)
#rssi data
rssi_df=RSSIDataLoader(Data_Path)

pour tester la performance de notre modèle, on test de mettre moins de récepteurs pour voir comment notre système se comporte.

In [20]:
map_gdf
print(map_gdf["geometry"][0])
print(type(map_gdf["geometry"][0]))

POLYGON ((2.199773490428924 48.71280859384019, 2.199789583683014 48.71287053721895, 2.1998579800128937 48.71286345797952, 2.1998432278633118 48.71280085091248, 2.199773490428924 48.71280859384019))
<class 'shapely.geometry.polygon.Polygon'>


In [18]:
m=MapCreation(map_gdf,location=[48,2],marker_gdf=marker_gdf)
m.save('output/map.html')
m

TypeError: Object of type Polygon is not JSON serializable

In [12]:
#MacModule à enlever

# deleted_mac_module=['C45BBE39870E','A8032A311DAA','A8032A312052','C4BBE384CD2','C45BBE37B346','A8032A30FB9E','A8032A311F6A','A8032A311F96','A8032A31204E','C45BBE384CD2','C45BBE38FEAA']
# def macModuleDeleter(marker_gdf,rssi_df,deleted_mac_module):
#     for mac_module in deleted_mac_module:
#         marker_gdf=marker_gdf[marker_gdf.macModule != mac_module]
#     for mac_module in deleted_mac_module:
#         rssi_df=rssi_df[rssi_df.macModule != mac_module]
#     return marker_gdf,rssi_df
# marker_gdf,rssi_df=macModuleDeleter(marker_gdf,rssi_df,deleted_mac_module)


In [13]:
m=MapCreation(map_gdf,marker_gdf=marker_gdf)
m



TypeError: Object of type Polygon is not JSON serializable

In [ ]:
#to optimize the getting of location of macModule, we create a dictionnary
MacModuleLocation = marker_gdf.set_index("macModule").to_dict()["geometry"]
#it display the approximation of point location in print but it is not the case in the dataframe

### Bind the received RSSI Data to the CartoModule Marker on Map

In [ ]:
#create a list of timestamps between min_timestamp and max_timestamp with a step of sampling_time
min_timestamp=rssi_df.iloc[0]['timestamp']+pd.Timedelta(sampling_time, unit="ms")
max_timestamp=rssi_df.iloc[-1]['timestamp']
timestamp_list= pd.date_range(start=min_timestamp, end=max_timestamp, freq=str(sampling_time)+"ms").tolist()


In [ ]:
MarkerDictList=getMarkerDictList(rssi_df, timestamp_list)
MarkerDictList


[{'A8032A311DAA': -66,
  'A8032A311F6A': -67,
  'A8032A311F96': -71,
  'A8032A31204A': -78,
  'A8032A311F56': -70,
  'A8032A311FAA': -78},
 {'A8032A311DAA': -66},
 {'A8032A311F56': -70,
  'A8032A311FAA': -77,
  'A8032A311F6A': -66,
  'A8032A311F96': -80},
 {'A8032A311F56': -79},
 {'A8032A311F6A': -69,
  'A8032A311F96': -73,
  'A8032A31204E': -78,
  'A8032A31204A': -76,
  'A8032A311F56': -67},
 {'A8032A311DAA': -66, 'A8032A31204E': -73},
 {'A8032A311F6A': -69, 'A8032A311F96': -70, 'A8032A31204E': -72},
 {'A8032A31204A': -72,
  'A8032A311F56': -65,
  'A8032A311FAA': -70,
  'A8032A311F96': -74,
  'A8032A311F6A': -65},
 {'A8032A311DAA': -78,
  'A8032A31204E': -72,
  'A8032A31204A': -75,
  'A8032A311F56': -78,
  'A8032A311FAA': -72},
 {'A8032A311F96': -68, 'A8032A311F6A': -57, 'A8032A311DAA': -79},
 {'A8032A311F56': -76, 'A8032A311DAA': -79, 'A8032A31204E': -68},
 {'A8032A311FAA': -79, 'A8032A311F96': -63, 'A8032A311F6A': -67},
 {'A8032A31204A': -76,
  'C45BBE384CD2': -79,
  'A8032A311F56':

colormap creation

In [ ]:
print(raw_data_features)

NameError: name 'raw_data_features' is not defined

In [ ]:
raw_data_features=RawDataFeatures(MarkerDictList,timestamp_list,MacModuleLocation)
MarkerLayer=TimestampedGeoJson(
                    {"type": "FeatureCollection","features": raw_data_features},
                    period='PT2S',
                    duration='PT1S',
                    transition_time=sampling_time,
                    auto_play=True)

In [ ]:
m=MapCreation(map_gdf,marker_gdf=marker_gdf)

MarkerLayer.add_to(m)
folium.LayerControl().add_to(m)
if saving_output:
    m.save('output/marker_map.html')
m

# implementation of low precision localisation

### 1. Naive implementation 

we're looking for 3 points at each sample,
if no 3 receivers received , we take the last 3 received,
if more than 3 receivers received we keep the 3 highest RSSI signal received

In [ ]:
def max_marker_dict(marker_dict,edge_number=3):
    #keep only the edge_number modules with the highest rssi values
    #if there is less than edge_number modules, we keep all of them

    if len(marker_dict) > edge_number:
        # to do : function only if number of received signal for the module in list is 1 
        marker_dict={key: marker_dict[key] for key in sorted(marker_dict, key=marker_dict.get, reverse=True)[:edge_number]}
    return(marker_dict)
def getPolygonList(MarkerDictList,MacModuleLocation,min_edge_number=3,max_edge_number=3):
    #works with polygon with moe than 3 points but not with 3 points because it is not a convex polygon, should use convex hull
    list_of_list_of_points=[]
    for i in range(len(MarkerDictList)):
        marker_dict=MarkerDictList[i]
        list_of_points = []
        marker_dict=max_marker_dict(marker_dict,max_edge_number)
        for mac_module,rssi in marker_dict.items():
            list_of_points.append([MacModuleLocation[mac_module].x,MacModuleLocation[mac_module].y])
        
        count_backward=1
        while len(list_of_points) < min_edge_number and i-count_backward >= 0:

            points_to_add=3-len(list_of_points)
            marker_dict=MarkerDictList[i-count_backward]
            marker_dict=max_marker_dict(marker_dict,points_to_add)

            
            for mac_module,rssi in marker_dict.items():
                location=[MacModuleLocation[mac_module].x,MacModuleLocation[mac_module].y]
                if location in list_of_points:
                    continue
                list_of_points.append(location)
            count_backward+=1
        list_of_list_of_points.append(list_of_points)
    #eject the marker not having the right number of points ( not triangle for example)
    for index,list1 in enumerate(list_of_list_of_points):
        if len(list1)<min_edge_number:
            print("error")
            list_of_list_of_points.pop(index)
    return(list_of_list_of_points)
list_of_list_of_points=getPolygonList(MarkerDictList,MacModuleLocation,min_edge_number=3,max_edge_number=3)


error


list_of_list_of_points est une liste définissant les points associés aux 3 récepteurs les plus forts pour chaque temps. on cherche maintenant à afficher les polygones associés dans une map temporelle

In [ ]:
def PolygonalFeatures(list_of_list_of_points,timestamp_list):
    input_format = '%Y-%m-%d %H:%M:%S.%f'
    output_format = '%Y-%m-%dT%H:%M:%S'
    features = []
    timestamp_list=[]
    for index in range(len(list_of_list_of_points)):
        timestamp = min_timestamp+index*pd.Timedelta(sampling_time, unit="ms")
        dt = datetime.strptime(str(timestamp), input_format)
        new_timestamp = dt.strftime(output_format)
        timestamp_list.append(new_timestamp)
    features = [{
        'type': 'Feature',
        'geometry': {
            'type': 'Polygon',
            'coordinates': list_of_list_of_points
        },
        'properties': {
            'times': timestamp_list,
            'style': {'color': 'red', 'weight': 1},
            
        }
    }]
    return features


In [ ]:
polygonal_features=PolygonalFeatures(list_of_list_of_points,timestamp_list)



m = MapCreation(map_gdf,marker_gdf=marker_gdf)


TimestampedGeoJson({"type": "FeatureCollection","features": polygonal_features},
                    period='PT2S',
                    duration='PT1S',
                    transition_time=sampling_time,  
                    auto_play=True).add_to(m)
folium.LayerControl().add_to(m)
m


In [ ]:
def polygonlist_to_polygoncenterlist(list_of_list_of_points):
    centerList=[]
    for list1 in list_of_list_of_points:
        n=0
        xlist=0
        ylist=0
        for point in list1:    
            xlist+=point[0]
            ylist+=point[1]
            n+=1
        centerList.append([xlist/n,ylist/n])
    return(centerList)
centerList=polygonlist_to_polygoncenterlist(list_of_list_of_points)
        

In [ ]:
centerList
for _center in centerList:
    if len(_center) != 2:
        print("error")

In [ ]:
def CenterOfPolygonsFeatures(centerList,list_of_list_of_points,sampling_time):
    #different timestamp list
    input_format = '%Y-%m-%d %H:%M:%S.%f'
    output_format = '%Y-%m-%dT%H:%M:%S'
    timestamp_list=[]
    for index in range(len(list_of_list_of_points)):
        timestamp = min_timestamp+index*pd.Timedelta(sampling_time, unit="ms")
        dt = datetime.strptime(str(timestamp), input_format)
        new_timestamp = dt.strftime(output_format)
        timestamp_list.append(new_timestamp)
        
    center_of_polygons_features = [{
        'type': 'Feature',
        'geometry': {
            'type': 'MultiPoint',
            'coordinates': centerList
        },
        'properties': {
            'times': timestamp_list,
            "icon": 'circle',
            'style': {'color': 'red', 'weight': 1},
            
        }
    }]
    return(center_of_polygons_features)
center_of_polygons_features=CenterOfPolygonsFeatures(centerList,list_of_list_of_points,sampling_time)
m = MapCreation(map_gdf,marker_gdf=marker_gdf)

TimestampedGeoJson(center_of_polygons_features,
                    period='PT2S',
                    duration='PT1S',
                    transition_time=sampling_time,  
                    auto_play=True).add_to(m)
folium.LayerControl().add_to(m)
if saving_output:
    m.save('output/map_with_point_locations.html')
m


In [ ]:

import numpy as np
number_of_points=len(centerList)
longitude_data=np.empty(number_of_points)
latitude_data=np.empty(number_of_points)
for index,point in enumerate(centerList):
    longitude_data[index]=point[0]
    latitude_data[index]=point[1]
latitude_data,longitude_data

(array([48.71318393, 48.71318148, 48.71318393, 48.71319069, 48.71317836,
        48.71317558, 48.71318148, 48.71316988, 48.71318109, 48.71317836,
        48.71317864, 48.71318499, 48.71317558, 48.71318215, 48.71317307,
        48.7131637 , 48.71315721, 48.71317101, 48.71318215, 48.71316988,
        48.71316726, 48.71318215, 48.71318586, 48.71316401, 48.71315402,
        48.71316972, 48.71317359, 48.71316164, 48.71315592, 48.71317649,
        48.71317359, 48.71317359, 48.71317077, 48.71316972, 48.71316169,
        48.71315592, 48.71315337, 48.71316164, 48.71316164, 48.71316164,
        48.71316458, 48.71316458, 48.71316458, 48.71313635, 48.71315205,
        48.71316028, 48.71314764, 48.71315908, 48.71314225, 48.71314225,
        48.71314225, 48.71313085, 48.71315592, 48.71315592, 48.71313085,
        48.7131447 , 48.7131447 , 48.71315336, 48.71313521, 48.71314483,
        48.7131379 , 48.71315175, 48.71313521, 48.71314225, 48.71315175,
        48.7131379 , 48.71314704, 48.71314238, 48.7

In [ ]:
def kalman_filter(data, Q, R):
    n = len(data)
    x_est = np.zeros(n)
    P = np.zeros(n)
    x_est[0] = data[0]
    P[0] = 1.0

    for i in range(1, n):
        x_pred = x_est[i-1]
        P_pred = P[i-1] + Q

        K = P_pred / (P_pred + R)
        x_est[i] = x_pred + K * (data[i] - x_pred)
        print("diff=",x_est[i]-x_pred)
        P[i] = (1 - K) * P_pred

    return x_est

Q = 30  # Process noise
R = 10  # Measurement noise


In [ ]:
def movingaveragetechnique(data,n=3):
    '''
    filtering the data with moving average technique


    https://en.wikipedia.org/wiki/Moving_average
    '''
    
    return np.convolve(data, np.ones(n), 'full') / n

In [ ]:
longitude_filtered = movingaveragetechnique(longitude_data,10)
latitude_filtered = movingaveragetechnique(latitude_data,10)
print(longitude_filtered)
centerList_filtered=np.vstack((longitude_filtered,latitude_filtered)).T.tolist()
#attention , problême ici, on a pas le même nombre de timestamp que de points
while len(timestamp_list)>len(centerList_filtered):
    timestamp_list.pop()
timestamp_list=[]
input_format = '%Y-%m-%d %H:%M:%S.%f'
output_format = '%Y-%m-%dT%H:%M:%S'
for index in range(len(list_of_list_of_points)):
    timestamp = min_timestamp+index*pd.Timedelta(sampling_time, unit="ms")
    dt = datetime.strptime(str(timestamp), input_format)
    new_timestamp = dt.strftime(output_format)
    timestamp_list.append(new_timestamp)
print(len(centerList_filtered),len(timestamp_list))
print(centerList_filtered)
print(timestamp_list)

[0.22006086 0.44012659 0.66018744 0.88025207 1.10031784 1.32038826
 1.54045399 1.76052962 1.98059295 2.20065872 2.20066607 2.20067018
 2.20067974 2.20068742 2.20069477 2.20070552 2.20072705 2.20073365
 2.20074264 2.20075249 2.20076246 2.20076492 2.20077077 2.200779
 2.20079565 2.20079705 2.20078939 2.20079744 2.20081337 2.20082206
 2.20082348 2.20083076 2.20083677 2.20083881 2.20083213 2.20083779
 2.20084884 2.20084884 2.20085088 2.20085685 2.20086507 2.2008733
 2.20087884 2.20089468 2.20090284 2.20091341 2.20091808 2.20092047
 2.20093056 2.20094066 2.20095324 2.20096872 2.20097586 2.2009724
 2.20098445 2.20098341 2.20098589 2.20099056 2.20099732 2.20100317
 2.20101601 2.20102043 2.20103262 2.20103805 2.20104247 2.20105792
 2.20106581 2.20107732 2.20107902 2.201086   2.201086   2.20108686
 2.20108857 2.20110903 2.20111928 2.20112071 2.2011392  2.20114357
 2.20115239 2.20115382 2.20116174 2.20117402 2.20118631 2.20118661
 2.20118187 2.2011734  2.2011523  2.20113156 2.2011199  2.2011141


In [ ]:
features = [{
    'type': 'Feature',
    'geometry': {
        'type': 'MultiPoint',
        'coordinates': centerList_filtered
    },
    'properties': {
        'times': timestamp_list,
        "icon": 'circle',
        'style': {'color': 'red', 'weight': 1},
        
    }
}]    
m = MapCreation(map_gdf,marker_gdf=marker_gdf)
TimestampedGeoJson(features,
                    period='PT2S',
                    duration='PT1S',
                    transition_time=sampling_time,  
                    auto_play=True).add_to(m)
folium.LayerControl().add_to(m)
if saving_output:
    m.save('output/map_with_point_locations_filtered.html')
m